# SensitiveDetector

通过 G4UserSteppingAction 中的函数 UserSteppingAction(const G4Step*) 可以获得模拟中每个 step 的信息。但是当我们的模拟阵列比较复杂，且我们感兴趣的探测器数量又比较少时，SensitiveDetector 提供一种机制，当模拟发生在我们指定的逻辑体内部时，我们才获取该 step 的信息。





## G4VSensitiveDetector

灵敏探测器继承抽象基类 G4VSensitiveDetector

![picsd1](img/G4VSensitiveDetector.png)


必须实现的纯虚函数为 G4bool ProcessHits(G4Step* aStep, G4TouchableHistory* ROhist)，可以简单将其理解为处理模拟的 step 数据。


```cpp
class wuSensitiveDetector : public G4VSensitiveDetector
{
public:
  wuSensitiveDetector(G4String name);
  virtual ~wuSensitiveDetector();

public:
  //  These two methods are invoked at the begining and at the end of each
  // event. The hits collection(s) created by this sensitive detector must
  // be set to the G4HCofThisEvent object at one of these two methods.
  void Initialize(G4HCofThisEvent*);
  void EndOfEvent(G4HCofThisEvent*);

protected: 
  //  The user MUST implement this method for generating hit(s) using the
  // information of G4Step object. Note that the volume and the position
  // information is kept in PreStepPoint of G4Step.
  //  Be aware that this method is a protected method and it sill be invoked
  // by Hit() method of Base class after Readout geometry associated to the
  // sensitive detector is handled.
  //  "ROhist" will be given only is a Readout geometry is defined to this
  // sensitive detector. The G4TouchableHistory object of the tracking geometry
  // is stored in the PreStepPoint object of G4Step.
  G4bool ProcessHits(G4Step*, G4TouchableHistory*);

  
private:
  wuHitCollection* hitsCollection;
  G4int HCID; //hits collection ID


private:

};
```



![picsd2](img/processhits.png)

**ProcessHits** 函数中，又是大家熟悉的 UserSteppingAction(const G4Step*) 中获得各种数据，这里我们同样将可能用到的数据全部罗列出来。在灵敏探测器中，通常将每个 step 的数据暂存在 G4VHit 中，等到当前 Event 模拟结束，再统一进行数据筛选以及数据存储。


## G4VHit

G4 中，提供 G4VHit 基类，用户继承该类来实现 step 所有信息的暂存。类 G4THitsCollection 实现了容器来存放 hit。

- Hit：记录 SD 中每个 G4Step 中的信息，因此一个 SD 中将产生多个 Hit 对象。
- HitCollection：一个 SD 中产生的 Hit，可以被记录在一个或者多个容器中， 该容器即为 HitCollection 对象。


![picsd3](img/wuhit.png)

数据的 Set/Get 函数，方便数据暂存以及读取。



## 定义灵敏探测器

G4VSensitiveDetector 对象可以被分配给 G4LogicalVolume，为了方便进行灵敏探测器的指定，我们在 **wuDetectorConstruction.cc** 中封装了一个函数，

```cpp
void wuDetectorConstruction::BuildSensitiveDetector(G4LogicalVolume* lv)
{
  if(!lv)
    {
      G4cout << "$$No logical volume, no SD built" << G4endl;
      return;
    }
  G4SDManager* SDman = G4SDManager::GetSDMpointer();
  wuSensitiveDetector* sd1 = new wuSensitiveDetector(lv->GetName()); //
  SDman->AddNewDetector(sd1);
  lv->SetSensitiveDetector(sd1);
}
```

在函数 **ConstructSDandField()** 中调用我们封装的函数来指定灵敏探测器。

```cpp
void wuDetectorConstruction::ConstructSDandField()
{
  //快速添加SD
  // BuildSensitiveDetector(logicWorld);

  // 如果采用 SD 输出，则关闭 step 输出

  BuildSensitiveDetector(logicDSSD);
  BuildSensitiveDetector(logicClover);
 
}
```

 

In [1]:
!date
!jupyter nbconvert SensitiveDetector.ipynb --to html

2024年 05月 19日 星期日 16:23:48 CST

[NbConvertApp] Converting notebook SensitiveDetector.ipynb to html

[NbConvertApp] Writing 569785 bytes to SensitiveDetector.html

